In [3]:
import pandas as pd
from sqlalchemy import create_engine
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)
import config

# USE FOR BOTH DIM AND FACT
class DataLoader:
    def __init__(self):
        self.storage_options = config.MINIO_CONFIG
    
    def read_parquet(self, path):
        return pd.read_parquet(path, storage_options=self.storage_options)
    
    def write_to_postgres(self, df, table_name, connection_string, if_exists='replace'):
        engine = create_engine(connection_string)
        df.to_sql(table_name, engine, if_exists='replace', index=False)

# Define input parameters
# Load to config file
parquet_path = 's3://tiki/curated/dim_product/ingestion_date=2024-04-06'
postgres_username = config.DATABASE_CONFIG['username']
postgres_password = config.DATABASE_CONFIG['password']
postgres_host = config.DATABASE_CONFIG['host']
postgres_port = config.DATABASE_CONFIG['port']
postgres_db = config.DATABASE_CONFIG['db_name']

table_name = 'dim_product'

# Create instance of DataLoader
data_processor = DataLoader()

# Read data from Parquet file
parquet_df = data_processor.read_parquet(parquet_path)

# Find the row with the maximum 'ingestion_dt_unix'
latest_row = parquet_df.loc[parquet_df['ingestion_dt_unix'].idxmax()]

# Filter DataFrame to get the latest data
latest_parquet = parquet_df[parquet_df['ingestion_dt_unix'] == latest_row['ingestion_dt_unix']]

# Define PostgreSQL connection string
connection_string = f'postgresql://{postgres_username}:{postgres_password}@{postgres_host}:{postgres_port}/{postgres_db}'

# Write the latest data to PostgreSQL
data_processor.write_to_postgres(latest_parquet, table_name, connection_string)

# Display the latest data
print(latest_parquet)


      tiki_pid                                               name brand_name  \
0    252608882  Tã/bỉm quần HUGGIES SKINCARE MEGA JUMBO size X...    Huggies   
1    252608891  Tã/bỉm quần HUGGIES SKINCARE gói SUPER JUMBO s...    Huggies   
2    210966404  Combo Sữa dưỡng thể Vaseline Gluta-Hya Dưỡng D...   Vaseline   
3    203853390                    Điện Thoại Oppo A17k (3GB/64GB)       OPPO   
4    200629270  [Tặng bình giữ nhiệt 1.5L]Combo 2 bịch cà phê ...    NESCAFÉ   
..         ...                                                ...        ...   
387    1453915  Ly Giữ Nhiệt Bằng Thép Không Gỉ Lock&Lock Clip...  LocknLock   
388    1672157  Bao cao su Durex Invisible Extra Thin Extra Se...      Durex   
389     897016  Combo Sân Chơi Trí Tuệ Của Chim Đa Đa (Bộ 6 Cuốn)       None   
390     579962  Bộ Nồi 5 Đáy Cao Cấp Núm Vàng Sunhouse SH781 Q...   Sunhouse   
391     414764  Lò Vi Sóng Sharp R-205VN(S) - 20L - Hàng chính...      Sharp   

         origin  ingestion_dt_unix  
0 